In [7]:
from cassandra.cluster import Cluster
import uuid

In [8]:
# you need to connect first using sudo cassandra -f otherwise connection err
cluster = Cluster(['cassandra']) # can support multiple IPs
session = cluster.connect()
keyspace = "ict"

# this is for cleaning
session.execute("DROP KEYSPACE IF EXISTS %s" % keyspace)


In [9]:
# create the ict keyspace
session.execute("""
        CREATE KEYSPACE %s
        WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
        """ % keyspace)

In [10]:
# use it 
session.set_keyspace(keyspace)

In [11]:
# remove the data stores if this already exists (perhaps a previous run)
session.execute("drop table if exists student")
session.execute("""
        CREATE TABLE student (
            studid uuid,
            name text,
            surname text,
            email text,
            PRIMARY KEY (studid)
        )
        """)

In [12]:
# uuid is generated from python module with same name
session.execute(
    """
    INSERT INTO student (name, surname, email, studid)
    VALUES (%s, %s, %s, %s)
    """,
    ("Yurgen", "Schembri", "ys@facebook.com", uuid.uuid1())
)

In [13]:
# get the student and pri
rows = session.execute('SELECT studid, name, surname, email FROM student')
for stud in rows:
    print(stud.name, stud.surname, stud.email)

Yurgen Schembri ys@facebook.com


In [14]:
## async

def handle_success(rows):
    stud = rows[0]
    print(stud.name, stud.surname, stud.studid)

def handle_error(exception):
    print(exception)

# sometimes (mostly the first time) nothing is shown here.  Why?  hint: async
future = session.execute_async('SELECT studid, name, surname, email FROM student')
future.add_callbacks(handle_success, handle_error)

Yurgen Schembri d9744e2a-1576-11e8-a114-0242ac110005


Doing a delete without specifying the primary key would mean Cassandra would have to first do the distributed search to get all of the records. Then internally issue the delete for all of those records. While it would be possible to implement, it is very different from the current write path inside Cassandra.  Good to know limitations.

In [16]:
session.execute("delete from student where name = 'Yurgen'") # this does not work, why?

InvalidRequest: Error from server: code=2200 [Invalid query] message="Some partition key parts are missing: studid"

In [17]:
result = session.execute("select * from student")
for r in result:
    print(r.studid)

d9744e2a-1576-11e8-a114-0242ac110005


In [18]:
session.execute("delete from student where studid = %s" % r.studid) # this works

In [19]:
result = session.execute("select * from student") # Darren, gone with the wind
for r in result:
    print(r.studid)